# 퍼셉트론
퍼셉트론은 엄연히 말하자면 딥러닝은 아니지만 딥러닝의 뉴런과 상당히 닮은 분류기이며, 뉴런의 계산 과정과 활성화 함수를 이해하기 위한 최적의 실습 예제이다.  
뉴런은 활성화함수를 자유롭게 선택할 수 있는 반면, 퍼셉트론은 스텝함수라는 활성화 함수만을 사용한다.

**뉴런의 출력값**: 활성화함수(가중치 * 입력값 + 편향값)  
**퍼셉트론의 출력값**: 스텝함수(가중치 * 입력값 + 편향값)

In [1]:
import tensorflow as tf

## 상수 설정
코드의 이해도를 높이기 위해 1은 True, 0은 False, 편향값(bias)은 1로 지정한다

In [2]:
T = 1.0
F = 0.0
bias = 1.0

## 데이터 획득

In [3]:
def get_AND_data():
    X = [
            [F, F, bias],
            [F, T, bias],
            [T, F, bias],
            [T, T, bias]
        ]
    y = [ [F],[F],[F],[T]]
    return X, y

def get_OR_data():
    X = [
            [F, F, bias],
            [F, T, bias],
            [T, F, bias],
            [T, T, bias]
        ]
    y = [ [F], [T], [T], [T]]
    return X, y

def get_XOR_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [T, T, bias]
    ]
    y = [ [F], [T], [T], [F]]
    return X, y

In [4]:
# 실습할 연산을 선택합니다.
X, y = get_AND_data()
# X, Y = get_OR_data()
# X, Y = get_XOR_data()

# 모델 생성
## 퍼셉트론 구현하기

논리 연상용 퍼셉트론을 구현해보자.  
논리 연산을 위한 입력값 X, Y와 편향값(b)을 받을 것이므로 weight를 [3,1]로 설정한다.  
3은 세개의 입력을 의미하고 1은 한 개의 뉴런임을 의미한다.

In [5]:
class Perceptron:
    def __init__(self):
        # 논리 연산을 위한 X, Y와 편향값(b)를 받을 것이므로, weight를 [3,1]로 설정한다
        # 3은 세개의 입력을 의미하고, 1는 한개의 뉴런임을 의미한다
        self.W = tf.Variable(tf.random.normal([3,1]))
        
    def train(self,X):
        err = 1
        epoch, max_epochs = 0,20
        while err > 0.0 and epoch < max_epochs:
            epoch += 1
            self.optimize(X)
            # MSE (평균제곱오차)를 관찰하며, 학습이 진행되는 동안 에러(MSE)가 줄어듬을 확인함
            err = self.mse(y, self.pred(X)).numpy()
            print('epoch:', epoch, 'mse:', err)
            

    @tf.function
    def faster_pred(self, X):
        return self.step(tf.matmul(X, self.W))
    
    def pred(self, X):
        return self.step(tf.matmul(X, self.W))
    
    def mse(self, y, y_hat):
        return tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
    
    # 퍼셉트론은 스텝 함수를 활성화함수로 사용한다. step(x) = {1 if x > 0; 0 otherwise}
    def step(self, x):
        #step(x) = {1 if x >0; 0 otherwise}
        return tf.dtypes.cast(tf.math.greater(x,0), tf.float32)
    
    def optimize(self, X):
        '''
        퍼셉트론은 경사하강법을 이용한 최적화가 불가능하다.
        매번 학습을 진행할 때마다 가중치를 아래의 룰에 맞게 업데이트한다.
        
        if target == 1 and activation == 0:
            w_new = w_old + input
        
        if target == 0 and activation == 1:
            w_new = w_old - input
        
        위의 두가지 조건은 아래의 코드로 간단히 구현 가능하다.
        '''
        delta = tf.matmul(X, tf.subtract(y, self.step(tf.matmul(X, self.W))), transpose_a=True)
        self.W.assign(self.W+delta)

## 학습

In [6]:
perceptron = Perceptron()
perceptron.train(X)

epoch: 1 mse: 0.25
epoch: 2 mse: 0.25
epoch: 3 mse: 0.25
epoch: 4 mse: 0.5
epoch: 5 mse: 0.25
epoch: 6 mse: 0.0


## 테스트

In [7]:
print(perceptron.pred(X).numpy())

[[0.]
 [0.]
 [0.]
 [1.]]


## tf.function 속도 비교

In [8]:
%timeit perceptron.pred(X).numpy()

91.8 µs ± 5.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%timeit perceptron.faster_pred(X).numpy()

137 µs ± 5.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
import timeit

In [11]:
pred_timeit = timeit.timeit(lambda: perceptron.pred(X).numpy(), number=100)
faster_pred_timeit = timeit.timeit(lambda: perceptron.faster_pred(X).numpy(), number=100)
time_diff = round(pred_timeit / faster_pred_timeit, 1)

In [12]:
print("faster_pred(@tf.function) is " + str(time_diff) + " times faster than pred!")

faster_pred(@tf.function) is 0.6 times faster than pred!
